In [64]:
import pandas as pd
import numpy as np
import os

def create_visit_area_mapping(df_visit, df_photo=None):
    """방문지 정보에서 장소명 기반으로 NEW_VISIT_AREA_ID 매핑 생성"""
    print("=== NEW_VISIT_AREA_ID 매핑 생성 ===")
    
    # 데이터 복사
    df = df_visit.copy()
    
    # 관광사진에만 있는 ID 추가
    if df_photo is not None:
        photo_only_ids = set(df_photo['VISIT_AREA_ID'].unique()) - set(df['VISIT_AREA_ID'].unique())
        if photo_only_ids:
            photo_unique = df_photo[df_photo['VISIT_AREA_ID'].isin(photo_only_ids)][['VISIT_AREA_ID', 'VISIT_AREA_NM']].drop_duplicates()
            for _, row in photo_unique.iterrows():
                if pd.notna(row['VISIT_AREA_NM']):
                    new_row = pd.Series({
                        'VISIT_AREA_ID': row['VISIT_AREA_ID'],
                        'VISIT_AREA_NM': row['VISIT_AREA_NM'],
                        'TRAVEL_ID': '',
                        'ROAD_NM_ADDR': '',
                        'LOTNO_ADDR': '',
                        'X_COORD': None,
                        'Y_COORD': None
                    })
                    df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
    
    def is_residence(name):
        """거주지 판별 - '집'만 체크 (횟집 등 제외)"""
        if not name:
            return False
        
        # 횟집, 술집 등은 제외
        exclude_patterns = ['횟집', '술집', '찜질', '커피', '볶는', '게스트']
        for pattern in exclude_patterns:
            if pattern in name:
                return False
        
        # '집' 관련 패턴
        residence_patterns = [
            name == '집',
            name.endswith(' 집'),
            name.endswith('집)'),
            '부모' in name and '집' in name,
            '친척' in name and '집' in name,
            '우리집' in name,
            '본가' in name,
            name.startswith('집 '),
        ]
        
        return any(residence_patterns)
    
    # 장소별 그룹화
    location_groups = {}
    
    for _, row in df.iterrows():
        visit_id = row['VISIT_AREA_ID']
        name = str(row['VISIT_AREA_NM']).strip() if pd.notna(row['VISIT_AREA_NM']) else f"이름없음_ID{visit_id}"
        
        # 거주지 통합
        if is_residence(name):
            key = "거주지(집_통합)"
        else:
            key = name
        
        if key not in location_groups:
            location_groups[key] = []
        location_groups[key].append(visit_id)
    
    # NEW_VISIT_AREA_ID 할당
    id_mapping = {}
    new_id = 1
    
    for location_key, visit_ids in sorted(location_groups.items()):
        for visit_id in visit_ids:
            id_mapping[visit_id] = new_id
        new_id += 1
    
    print(f"총 {len(id_mapping)}개 ID를 {new_id-1}개 NEW_ID로 매핑")
    
    # 결과 DataFrame 생성
    df['NEW_VISIT_AREA_ID'] = df['VISIT_AREA_ID'].map(id_mapping)
    
    return df, id_mapping

def main():
    """메인 실행 함수"""
    try:
        # 데이터 로드
        print("=== 데이터 로드 ===")
        df_visit = pd.read_csv('./merged_csv/방문지_total.csv')
        df_photo = pd.read_csv('./merged_csv/관광사진_total.csv')
        df_move = pd.read_csv('./merged_csv/fin/이동내역_fin.csv')
        
        print(f"방문지: {len(df_visit)}행, 관광사진: {len(df_photo)}행, 이동내역: {len(df_move)}행")
        
        # NEW_VISIT_AREA_ID 매핑 생성
        df_visit_updated, id_mapping = create_visit_area_mapping(df_visit, df_photo)
        
        # 관광사진 데이터 업데이트
        df_photo_updated = df_photo.copy()
        df_photo_updated['NEW_VISIT_AREA_ID'] = df_photo_updated['VISIT_AREA_ID'].map(id_mapping)
        
        # 이동내역 데이터 업데이트
        df_move_updated = df_move.copy()
        df_move_updated['NEW_START_VISIT_AREA_ID'] = df_move_updated['START_VISIT_AREA_ID'].map(id_mapping)
        df_move_updated['NEW_END_VISIT_AREA_ID'] = df_move_updated['END_VISIT_AREA_ID'].map(id_mapping)
        
        # 결과 저장
        print("\n=== 결과 저장 ===")
        os.makedirs('./merged_csv/fin', exist_ok=True)
        
        df_visit_updated.to_csv('./merged_csv/fin/방문지_fin.csv', index=False, encoding='utf-8-sig')
        df_photo_updated.to_csv('./merged_csv/fin/관광사진_fin.csv', index=False, encoding='utf-8-sig')
        df_move_updated.to_csv('./merged_csv/fin/이동내역_fin.csv', index=False, encoding='utf-8-sig')
        
        print("저장 완료!")
        print(f"NEW_VISIT_AREA_ID 범위: 1 ~ {max(id_mapping.values())}")
        
        return df_visit_updated, df_photo_updated, df_move_updated, id_mapping
        
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

if __name__ == "__main__":
    main()

=== 데이터 로드 ===
방문지: 140555행, 관광사진: 66508행, 이동내역: 140555행
=== NEW_VISIT_AREA_ID 매핑 생성 ===
총 2387개 ID를 46799개 NEW_ID로 매핑

=== 결과 저장 ===
저장 완료!
NEW_VISIT_AREA_ID 범위: 1 ~ 46799


# 매핑 파일 기반으로 매핑 진행

In [88]:
import os
import pandas as pd

# 매핑 결과 파일 로드
mapping_df = pd.read_csv('./merged_csv/fin/방문지_fin.csv')
mapping_df['TRAVEL_ID'] = mapping_df['TRAVEL_ID'].fillna('').astype(str)
mapping_df['VISIT_AREA_ID'] = mapping_df['VISIT_AREA_ID'].astype(str)

# 매핑 딕셔너리 생성
pair_to_new_id = mapping_df.set_index(['TRAVEL_ID', 'VISIT_AREA_ID'])['NEW_VISIT_AREA_ID'].to_dict()
name_to_new_id = mapping_df.set_index('VISIT_AREA_NM')['NEW_VISIT_AREA_ID'].to_dict()
id_to_new_id = mapping_df.set_index('VISIT_AREA_ID')['NEW_VISIT_AREA_ID'].to_dict()
# 핵심: ID -> 장소명 매핑 (이동내역용)
id_to_name = mapping_df.set_index('VISIT_AREA_ID')['VISIT_AREA_NM'].to_dict()

data_dir = './merged_csv/merged_csv_region/'
region_list = [r for r in os.listdir(data_dir) if not r.startswith('.') and os.path.isdir(os.path.join(data_dir, r))]

for region in region_list:
    print(f"\n=== {region} 처리 시작 ===")
    region_path = os.path.join(data_dir, region)
    region_output_dir = os.path.join(region_path, 'fin')
    os.makedirs(region_output_dir, exist_ok=True)

    # 1️⃣ 방문지 정보 처리
    visit_file = os.path.join(region_path, '방문지_total.csv')
    if os.path.exists(visit_file):
        df_visit = pd.read_csv(visit_file)
        df_visit['TRAVEL_ID'] = df_visit['TRAVEL_ID'].fillna('').astype(str)
        df_visit['VISIT_AREA_ID'] = df_visit['VISIT_AREA_ID'].astype(str)
        
        # 매핑 시도 (우선순위: (TRAVEL_ID, VISIT_AREA_ID) -> VISIT_AREA_ID -> VISIT_AREA_NM)
        new_ids = []
        for _, row in df_visit.iterrows():
            key1 = (row['TRAVEL_ID'], row['VISIT_AREA_ID'])
            key2 = row['VISIT_AREA_ID']
            key3 = row['VISIT_AREA_NM']
            
            if key1 in pair_to_new_id:
                new_ids.append(pair_to_new_id[key1])
            elif key2 in id_to_new_id:
                new_ids.append(id_to_new_id[key2])
            elif key3 in name_to_new_id:
                new_ids.append(name_to_new_id[key3])
            else:
                new_ids.append(None)  # 매핑 실패
        
        df_visit['NEW_VISIT_AREA_ID'] = new_ids
        save_path = os.path.join(region_output_dir, '방문지_fin.csv')
        df_visit.to_csv(save_path, index=False, encoding='utf-8-sig')
        
        unmapped = df_visit['NEW_VISIT_AREA_ID'].isna().sum()
        print(f"{region} 방문지_fin.csv 저장 (shape: {df_visit.shape}, 매핑실패: {unmapped})")

    # 2️⃣ 관광사진 정보 처리
    photo_file = os.path.join(region_path, '관광사진_total.csv')
    if os.path.exists(photo_file):
        df_photo = pd.read_csv(photo_file)
        df_photo['TRAVEL_ID'] = df_photo['TRAVEL_ID'].fillna('').astype(str)
        df_photo['VISIT_AREA_ID'] = df_photo['VISIT_AREA_ID'].astype(str)
        
        new_ids = []
        for _, row in df_photo.iterrows():
            key1 = (row['TRAVEL_ID'], row['VISIT_AREA_ID'])
            key2 = row['VISIT_AREA_ID']
            key3 = row['VISIT_AREA_NM']
            
            if key1 in pair_to_new_id:
                new_ids.append(pair_to_new_id[key1])
            elif key2 in id_to_new_id:
                new_ids.append(id_to_new_id[key2])
            elif key3 in name_to_new_id:
                new_ids.append(name_to_new_id[key3])
            else:
                new_ids.append(None)  # 매핑 실패
        
        df_photo['NEW_VISIT_AREA_ID'] = new_ids
        save_path = os.path.join(region_output_dir, '관광사진_fin.csv')
        df_photo.to_csv(save_path, index=False, encoding='utf-8-sig')
        
        unmapped = df_photo['NEW_VISIT_AREA_ID'].isna().sum()
        print(f"{region} 관광사진_fin.csv 저장 (shape: {df_photo.shape}, 매핑실패: {unmapped})")

    # 3️⃣ 이동내역 정보 처리
    move_file = os.path.join(region_path, 'fin', '이동내역_fin.csv')
    if os.path.exists(move_file):
        df_move = pd.read_csv(move_file)
        df_move['TRAVEL_ID'] = df_move['TRAVEL_ID'].fillna('').astype(str)
        
        start_new_ids = []
        end_new_ids = []
        
        for _, row in df_move.iterrows():
            trip_id = str(int(row['TRIP_ID'])) if pd.notna(row['TRIP_ID']) else None
            
            if pd.notna(row['START_VISIT_AREA_ID']):  # START가 있는 경우 (첫 목적지)
                # TRIP_ID를 이용해서 매핑
                if trip_id and trip_id in id_to_name:
                    start_name = id_to_name[trip_id]
                    start_new_ids.append(name_to_new_id.get(start_name))
                else:
                    start_new_ids.append(None)
                end_new_ids.append(None)  # END는 NaN
                
            elif pd.notna(row['END_VISIT_AREA_ID']):  # END가 있는 경우 (경유지)
                # TRIP_ID를 이용해서 매핑
                if trip_id and trip_id in id_to_name:
                    end_name = id_to_name[trip_id]
                    end_new_ids.append(name_to_new_id.get(end_name))
                else:
                    end_new_ids.append(None)
                start_new_ids.append(None)  # START는 NaN
                
            else:  # 둘 다 없는 경우
                start_new_ids.append(None)
                end_new_ids.append(None)
        
        df_move['NEW_START_VISIT_AREA_ID'] = start_new_ids
        df_move['NEW_END_VISIT_AREA_ID'] = end_new_ids
        
        save_path = os.path.join(region_output_dir, '이동내역_fin.csv')
        df_move.to_csv(save_path, index=False, encoding='utf-8-sig')
        
        start_count = sum(1 for x in start_new_ids if pd.notna(x))
        end_count = sum(1 for x in end_new_ids if pd.notna(x))
        start_unmapped = sum(1 for i, row in df_move.iterrows() if pd.notna(row['START_VISIT_AREA_ID']) and pd.isna(start_new_ids[i]))
        end_unmapped = sum(1 for i, row in df_move.iterrows() if pd.notna(row['END_VISIT_AREA_ID']) and pd.isna(end_new_ids[i]))
        
        print(f"{region} 이동내역_fin.csv 저장 (shape: {df_move.shape})")
        print(f"  - START 매핑: {start_count}개 성공, {start_unmapped}개 실패")
        print(f"  - END 매핑: {end_count}개 성공, {end_unmapped}개 실패")

print("\n=== 모든 지역 처리 완료 ===")


=== 서부권 처리 시작 ===
서부권 방문지_fin.csv 저장 (shape: (31875, 26), 매핑실패: 0)
서부권 관광사진_fin.csv 저장 (shape: (16549, 15), 매핑실패: 0)
서부권 이동내역_fin.csv 저장 (shape: (31875, 15))
  - START 매핑: 2883개 성공, 0개 실패
  - END 매핑: 28992개 성공, 0개 실패

=== 동부권 처리 시작 ===
동부권 방문지_fin.csv 저장 (shape: (32930, 26), 매핑실패: 0)
동부권 관광사진_fin.csv 저장 (shape: (16266, 15), 매핑실패: 0)
동부권 이동내역_fin.csv 저장 (shape: (32930, 15))
  - START 매핑: 2882개 성공, 0개 실패
  - END 매핑: 30048개 성공, 0개 실패

=== 제주도 및 도서지역 처리 시작 ===
제주도 및 도서지역 방문지_fin.csv 저장 (shape: (51596, 26), 매핑실패: 0)
제주도 및 도서지역 관광사진_fin.csv 저장 (shape: (17224, 15), 매핑실패: 0)
제주도 및 도서지역 이동내역_fin.csv 저장 (shape: (51596, 15))
  - START 매핑: 2881개 성공, 0개 실패
  - END 매핑: 48715개 성공, 0개 실패

=== 수도권 처리 시작 ===
수도권 방문지_fin.csv 저장 (shape: (24154, 26), 매핑실패: 0)
수도권 관광사진_fin.csv 저장 (shape: (16469, 15), 매핑실패: 0)
수도권 이동내역_fin.csv 저장 (shape: (24154, 15))
  - START 매핑: 2883개 성공, 0개 실패
  - END 매핑: 21271개 성공, 0개 실패

=== 모든 지역 처리 완료 ===


In [86]:
move_test = pd.read_csv('/Users/idaeho/Documents/GitHub/GNN_Recommend/process_total/merged_csv/merged_csv_region/서부권/fin/이동내역_fin.csv')
move_test.head()

,Unnamed: 0,TRAVEL_ID,TRIP_ID,START_VISIT_AREA_ID,END_VISIT_AREA_ID,START_DT_MIN,END_DT_MIN,MVMN_CD_1,MVMN_CD_2,REGION,DATA_TYPE,NEW_START_VISIT_AREA_ID,NEW_END_VISIT_AREA_ID,START_VISIT_AREA_NM,END_VISIT_AREA_NM
0,0,e_e000005,2304290001,2.304290e+09,NaN,2023-04-29 12:00,NaN,0.0,0.0,서부권,_csv_train,46754.0,NaN,NaN,NaN
1,1,e_e000005,2304290002,NaN,2.304290e+09,NaN,2023-04-29 12:30,1.0,0.0,서부권,_csv_train,NaN,46754.0,NaN,NaN
2,2,e_e000005,2304290003,NaN,2.304290e+09,NaN,2023-04-29 13:30,1.0,0.0,서부권,_csv_train,NaN,46469.0,NaN,NaN
3,3,e_e000005,2304290004,NaN,2.304290e+09,NaN,2023-04-29 16:30,1.0,0.0,서부권,_csv_train,NaN,45779.0,NaN,NaN
4,4,e_e000005,2304290005,NaN,2.304290e+09,NaN,2023-04-29 17:00,5.0,0.0,서부권,_csv_train,NaN,42684.0,NaN,NaN
